In [77]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog,messagebox,simpledialog
import pandas as pd
import numpy as np
from pandastable import Table
import os

janela = Tk()
janela.title("Editor de Excel com Pandas")  
janela.attributes("-fullscreen", True)

class ExcelEditor:
    def __init__(self, janela_principal):
        self.janela_principal = janela_principal
        
        self.resultado_label = Label(self.janela_principal,text="Total: ",
                                    font=("Arial",16),bg="#f5f5f5")
        self.resultado_label.pack(side=TOP,padx=10,pady=10)

        self.df = pd.DataFrame()

        self.tree = None
        self.table = None
        self.filename = ""

        self.cria_widgets()

        self.tree.bind("<Double-1>",self.editarItens)

    def cria_widgets(self):
        menu_bar = tk.Menu(self.janela_principal)
        
        menu_de_arquivo = tk.Menu(menu_bar, tearoff=0)
        menu_de_arquivo.add_command(label="Abrir",command=self.carregar_excel)
        menu_de_arquivo.add_separator()
        menu_de_arquivo.add_command(label="Salvar Como",command=self.salvar_como)
        menu_de_arquivo.add_separator()
        menu_de_arquivo.add_command(label="Sair",command=janela.destroy)

        menu_bar.add_cascade(label="Arquivo", menu=menu_de_arquivo)

        #----------------------------------------------------------------------------

        menu_edicao = tk.Menu(menu_bar, tearoff=0)
        menu_edicao.add_command(label="Renomear Coluna",command=self.renomear_coluna)
        menu_edicao.add_command(label="Remover Coluna",command=self.remover_coluna)
        menu_edicao.add_command(label="Filtrar",command=self.filtrar_coluna)
        menu_edicao.add_command(label="Pivot",command=janela.destroy)
        menu_edicao.add_command(label="Group",command=self.agrupar_coluna)
        menu_edicao.add_command(label="Remover Linhas em Branco",command=self.remover_linhas_em_branco)        
        menu_edicao.add_command(label="Remover Linhas Alternadas",command=self.remove_algumas_linhas)
        menu_edicao.add_command(label="Remover Duplicados",command=self.remover_duplicados)

        menu_bar.add_cascade(label="Editar", menu=menu_edicao)

        #----------------------------------------------------------------------------

        marge_menu = tk.Menu(menu_bar, tearoff=0)
        marge_menu.add_command(label="Inner Join",command=self.merge_inner_join)
        marge_menu.add_command(label="Join Full",command=self.merge_full_join)
        marge_menu.add_command(label="Left Join",command=self.merge_left_join)
        marge_menu.add_command(label="Marge Outer",command=self.merge_outer)

        menu_bar.add_cascade(label="Merge", menu=marge_menu)

        #----------------------------------------------------------------------------

        relatorio_menu = tk.Menu(menu_bar, tearoff=0)
        relatorio_menu.add_command(label="Consolidar",command=self.consolidar_arquivos)
        relatorio_menu.add_command(label="Quebra",command=self.quebrar_arquivo)

        menu_bar.add_cascade(label="Relatório", menu=relatorio_menu)

        self.janela_principal.config(menu=menu_bar)

        self.tree = tk.ttk.Treeview(self.janela_principal)

        self.tree.pack(expand=False)

    def soma_colunas_com_valor(self):
        resultados = []
        for coluna in self.df.columns:
            if pd.api.types.is_numeric_dtype(self.df[coluna]):
                valores_numericos = self.df[coluna][0:]
                valores_numericos = pd.to_numeric(valores_numericos,errors='coerce')
                valores_numericos = valores_numericos[~np.isnan(valores_numericos)]

                soma = valores_numericos.sum()

                resultado = f"A soma da coluna {coluna} é {soma}"
                
                resultados.append(resultado)

        self.resultado_label.config(text="\n".join(resultados))

    def carregar_excel(self):
        tipo_de_arquivo = (("Excel files","*.xlsx;*.xls"),("All files","*.*"))

        self.nome_do_arquivo = filedialog.askopenfilename(title="Selecione o arquivo",
                                                        filetypes=tipo_de_arquivo)
        
        try:
            self.df = pd.read_excel(self.nome_do_arquivo)
            self.atualiza_treeview()

        except Exception as e:
            messagebox.showerror("Erro",f"Não foi possivel abrir o arquivo: {e}")

        self.soma_colunas_com_valor()
                
    def atualiza_treeview(self):
        self.tree.delete(*self.tree.get_children())
        self.tree["columns"] = list(self.df.columns)

        for column in self.df.columns:
            self.tree.heading(column,text=column)

        for i, row in self.df.iterrows():
            values = list(row)
            for j, value in enumerate(values):
                if isinstance(value, np.generic):
                    values[j] = np.asscalar(value)

            self.tree.insert("",tk.END,values=values)

    def renomear_coluna(self):
        janela_renomear_coluna = tk.Toplevel(self.janela_principal)
        janela_renomear_coluna.title("Renomear Coluna")

        largura_janela = 400
        altura_janela = 250

        largura_tela = janela_renomear_coluna.winfo_screenwidth()
        altura_tela = janela_renomear_coluna.winfo_screenheight()

        pos_x = (largura_tela // 2) - (largura_janela // 2)
        pos_y = (altura_tela // 2) - (altura_janela // 2)

        janela_renomear_coluna.geometry(f"{largura_janela}x{altura_janela}+{pos_x}+{pos_y}")

        janela_renomear_coluna.configure(bg="#ffffff")
        
        label_coluna = tk.Label(janela_renomear_coluna,text="Digite o nome da coluna que deseja renomear: ",
                                font=("Arial",12),bg="#ffffff")
        label_coluna.pack(pady=10)
        entry_coluna = tk.Entry(janela_renomear_coluna,font=("Arial",12))
        entry_coluna.pack()
        
        label_novo_nome = tk.Label(janela_renomear_coluna,text="Digite o novo nome: ",
                                font=("Arial",12),bg="#ffffff")
        label_novo_nome.pack(pady=10)
        entry_novo_nome = tk.Entry(janela_renomear_coluna,font=("Arial",12))
        entry_novo_nome.pack()

        botao_renomear = tk.Button(janela_renomear_coluna,text="Renomear",
                                font=("Arial",12), command=lambda: self.funcao_renomear_coluna(entry_coluna.get(),entry_novo_nome.get(),janela_renomear_coluna))
        botao_renomear.pack(pady=20)

        janela_renomear_coluna.mainloop()

    def funcao_renomear_coluna(self,coluna,novo_nome,janela_renomear_coluna):
        if novo_nome:
            self.df = self.df.rename(columns={coluna:novo_nome})
            self.atualiza_treeview()
            janela_renomear_coluna.destroy()

    def remover_linhas_em_branco(self):
        resposta = messagebox.askyesno("Remover linhas em branco","Tem certeza que deseja remover as linhas em branco?")

        if resposta == 1:
            self.df = self.df.dropna(axis=0)
            self.atualiza_treeview()
            self.soma_colunas_com_valor()

    def remove_algumas_linhas(self):
        janela_remove_algumas_linha = tk.Toplevel(self.janela_principal)
        janela_remove_algumas_linha.title("Remover Algumas Linhas")

        largura_janela = 400
        altura_janela = 250

        largura_tela = janela_remove_algumas_linha.winfo_screenwidth()
        altura_tela = janela_remove_algumas_linha.winfo_screenheight()

        pos_x = (largura_tela // 2) - (largura_janela // 2)
        pos_y = (altura_tela // 2) - (altura_janela // 2)

        janela_remove_algumas_linha.geometry(f"{largura_janela}x{altura_janela}+{pos_x}+{pos_y}")
        janela_remove_algumas_linha.configure(bg="#ffffff")
        
        label_linha_inicio = tk.Label(janela_remove_algumas_linha,text="Digite o número da primeira linha que deseja remover: ",
                                font=("Arial",12),bg="#ffffff")
        label_linha_inicio.pack(pady=10)
        entry_linha_inicio = tk.Entry(janela_remove_algumas_linha,font=("Arial",12))
        entry_linha_inicio.pack()
            
        label_linha_fim = tk.Label(janela_remove_algumas_linha,text="Digite o número da última linha que deseja remover: ",
                                font=("Arial",12),bg="#ffffff")
        label_linha_fim.pack(pady=10)
        entry_linha_fim = tk.Entry(janela_remove_algumas_linha,font=("Arial",12))
        entry_linha_fim.pack()

        botao_remover = tk.Button(janela_remove_algumas_linha,text="Remover",
                                font=("Arial",12), 
                                command=lambda: 
                                self.funcao_remover_algumas_linhas(entry_linha_inicio.get(),
                                                            entry_linha_fim.get(),
                                                            janela_remove_algumas_linha))
        botao_remover.pack(pady=20)

        janela_remove_algumas_linha.mainloop()

    def funcao_remover_algumas_linhas(self,linha_inicio,linha_fim,janela_remove_algumas_linha):
        
        primeira_linha = int(linha_inicio)
        ultima_linha = int(linha_fim)

        self.df = self.df.drop(self.df.index[primeira_linha-1:ultima_linha-1])
        self.atualiza_treeview()
        self.soma_colunas_com_valor()
        janela_remove_algumas_linha.destroy()

    def remover_duplicados(self):
        janela_remove_duplicados = tk.Toplevel(self.janela_principal)
        janela_remove_duplicados.title("Remover duplicados")

        largura_janela = 500
        altura_janela = 150

        largura_tela = janela_remove_duplicados.winfo_screenwidth()
        altura_tela = janela_remove_duplicados.winfo_screenheight()

        pos_x = (largura_tela // 2) - (largura_janela // 2)
        pos_y = (altura_tela // 2) - (altura_janela // 2)

        janela_remove_duplicados.geometry(f"{largura_janela}x{altura_janela}+{pos_x}+{pos_y}")
        janela_remove_duplicados.configure(bg="#ffffff")
        
        label_coluna = tk.Label(janela_remove_duplicados,text="Digite o nome da coluna que deseja remover os itens duplicados: ",
                                font=("Arial",12),bg="#ffffff")
        label_coluna.pack(pady=10)
        entry_coluna = tk.Entry(janela_remove_duplicados,font=("Arial",12))
        entry_coluna.pack()            

        botao_remover = tk.Button(janela_remove_duplicados,text="Remover",
                                font=("Arial",12), 
                                command=lambda: 
                                self.funcao_remover_duplicados(entry_coluna.get(),
                                                            janela_remove_duplicados))
        botao_remover.pack(pady=20)

        janela_remove_duplicados.mainloop()

    def funcao_remover_duplicados(self,coluna,janela_remove_duplicados):
        if coluna:
            self.df = self.df.drop_duplicates(subset=coluna,keep="first")
            
        self.atualiza_treeview()
        self.soma_colunas_com_valor()
        janela_remove_duplicados.destroy()
    
    def remover_coluna(self):
        janela_remove_coluna = tk.Toplevel(self.janela_principal)
        janela_remove_coluna.title("Remover Coluna")

        largura_janela = 500
        altura_janela = 150

        largura_tela = janela_remove_coluna.winfo_screenwidth()
        altura_tela = janela_remove_coluna.winfo_screenheight()

        pos_x = (largura_tela // 2) - (largura_janela // 2)
        pos_y = (altura_tela // 2) - (altura_janela // 2)

        janela_remove_coluna.geometry(f"{largura_janela}x{altura_janela}+{pos_x}+{pos_y}")
        janela_remove_coluna.configure(bg="#ffffff")
        
        label_coluna = tk.Label(janela_remove_coluna,text="Digite o nome da coluna que deseja remover: ",
                                font=("Arial",12),bg="#ffffff")
        label_coluna.pack(pady=10)
        entry_coluna = tk.Entry(janela_remove_coluna,font=("Arial",12))
        entry_coluna.pack()            

        botao_remover = tk.Button(janela_remove_coluna,text="Remover",
                                font=("Arial",12), 
                                command=lambda: 
                                self.funcao_remover_coluna(entry_coluna.get(),
                                                            janela_remove_coluna))
        botao_remover.pack(pady=20)

        janela_remove_coluna.mainloop()

    def funcao_remover_coluna(self,coluna,janela_remove_coluna):
        if coluna:
            self.df = self.df.drop(columns=coluna)
            
        self.atualiza_treeview()
        self.soma_colunas_com_valor()
        janela_remove_coluna.destroy()

    def filtrar_coluna(self):
        janela_filtrar_coluna = tk.Toplevel(self.janela_principal)
        janela_filtrar_coluna.title("Filtrar Coluna")

        largura_janela = 400
        altura_janela = 250

        largura_tela = janela_filtrar_coluna.winfo_screenwidth()
        altura_tela = janela_filtrar_coluna.winfo_screenheight()

        pos_x = (largura_tela // 2) - (largura_janela // 2)
        pos_y = (altura_tela // 2) - (altura_janela // 2)

        janela_filtrar_coluna.geometry(f"{largura_janela}x{altura_janela}+{pos_x}+{pos_y}")

        janela_filtrar_coluna.configure(bg="#ffffff")
        
        label_coluna = tk.Label(janela_filtrar_coluna,text="Digite o nome da coluna que deseja filtrar: ",
                                font=("Arial",12),bg="#ffffff")
        label_coluna.pack(pady=10)
        entry_coluna = tk.Entry(janela_filtrar_coluna,font=("Arial",12))
        entry_coluna.pack()
        
        label_valor_filtro = tk.Label(janela_filtrar_coluna,text="Digite o valor a ser filtrado: ",
                                font=("Arial",12),bg="#ffffff")
        label_valor_filtro.pack(pady=10)
        entry_valor_filtro = tk.Entry(janela_filtrar_coluna,font=("Arial",12))
        entry_valor_filtro.pack()

        botao_renomear = tk.Button(janela_filtrar_coluna,text="Filtrar",
                                font=("Arial",12), 
                                command=lambda: 
                                self.funcao_filtrar_coluna(
                                    entry_coluna.get(),
                                    entry_valor_filtro.get(),
                                    janela_filtrar_coluna))
        botao_renomear.pack(pady=20)

        janela_filtrar_coluna.mainloop()

    def funcao_filtrar_coluna(self,coluna,valor_filtro,janela_filtrar_coluna):
        if coluna:
            self.df = self.df[self.df[coluna] == valor_filtro]

        self.atualiza_treeview()
        self.soma_colunas_com_valor()
        janela_filtrar_coluna.destroy()
    
    def agrupar_coluna(self):
        janela_agrupar_coluna = tk.Toplevel(self.janela_principal)
        janela_agrupar_coluna.title("Agrupar por Coluna")

        largura_janela = 500
        altura_janela = 150

        largura_tela = janela_agrupar_coluna.winfo_screenwidth()
        altura_tela = janela_agrupar_coluna.winfo_screenheight()

        pos_x = (largura_tela // 2) - (largura_janela // 2)
        pos_y = (altura_tela // 2) - (altura_janela // 2)

        janela_agrupar_coluna.geometry(f"{largura_janela}x{altura_janela}+{pos_x}+{pos_y}")
        janela_agrupar_coluna.configure(bg="#ffffff")
        
        label_coluna = tk.Label(janela_agrupar_coluna,text="Digite o nome da coluna que deseja agrupar: ",
                                font=("Arial",12),bg="#ffffff")
        label_coluna.pack(pady=10)
        entry_coluna = tk.Entry(janela_agrupar_coluna,font=("Arial",12))
        entry_coluna.pack()            

        botao_remover = tk.Button(janela_agrupar_coluna,text="Agrupar",
                                font=("Arial",12), 
                                command=lambda: 
                                self.funcao_agrupar_coluna(entry_coluna.get(),
                                                            janela_agrupar_coluna))
        botao_remover.pack(pady=20)

        janela_agrupar_coluna.mainloop()

    def funcao_agrupar_coluna(self,coluna,janela_agrupar_coluna):
        self.tree.delete(*self.tree.get_children())

        if coluna:
            dados_agrupados = self.df.groupby(coluna).sum()

            for i, linha in dados_agrupados.iterrows():
                values = list(linha)
                
                for j, value in enumerate(values):
                    if isinstance(value, np.generic):
                        values[j] = np.asscalar(value)

                self.tree.insert("",tk.END,values=[i]+values)

        janela_agrupar_coluna.destroy()

    def merge_inner_join(self):
        tipo_de_arquivo = (("Excel files","*.xlsx;*.xls"),("All files","*.*"))

        arquivo1 = filedialog.askopenfilename(title="Selecione o primeiro arquivo",
                                                        filetypes=tipo_de_arquivo)
        arquivo2 = filedialog.askopenfilename(title="Selecione o segundo arquivo",
                                                        filetypes=tipo_de_arquivo) 

        arquivo1 = pd.read_excel(arquivo1)
        arquivo2 = pd.read_excel(arquivo2)

        coluna_join = simpledialog.askstring("Coluna do Inner Join","Digite o nome da coluna que será utilizada para o Inner Join: ")
        
        #self.df = pd.merge(arquivo1,arquivo2,on=coluna_join, how="inner")
        #self.df = pd.merge(arquivo1,arquivo2[["Vendedor","Total Vendas"]],on=coluna_join, how="inner",suffixes=(" Loja 1"," Loja 2"))
        self.df = pd.merge(arquivo1,arquivo2[["Vendedor","Total Vendas"]],on=coluna_join, how="inner",suffixes=(" Loja 1"," Loja 2"))

        self.df = self.df[["Vendedor","Total Vendas Loja 1","Total Vendas Loja 2"]]

        self.atualiza_treeview()
        self.soma_colunas_com_valor()

    def merge_full_join(self):
        tipo_de_arquivo = (("Excel files","*.xlsx;*.xls"),("All files","*.*"))

        arquivo1 = filedialog.askopenfilename(title="Selecione o primeiro arquivo",
                                                        filetypes=tipo_de_arquivo)
        arquivo2 = filedialog.askopenfilename(title="Selecione o segundo arquivo",
                                                        filetypes=tipo_de_arquivo) 

        arquivo1 = pd.read_excel(arquivo1)
        arquivo2 = pd.read_excel(arquivo2)

        #coluna_join = simpledialog.askstring("Coluna do Full Join","Digite o nome da coluna que será utilizada para o Full Join: ")
        
        #self.df = pd.merge(arquivo1,arquivo2,on=coluna_join, how="inner")
        #self.df = pd.merge(arquivo1,arquivo2[["Vendedor","Total Vendas"]],on=coluna_join, how="inner",suffixes=(" Loja 1"," Loja 2"))
        #self.df = pd.merge(arquivo1,arquivo2[["Vendedor","Total Vendas"]],on=coluna_join, how="",suffixes=(" Loja 1"," Loja 2"))
        #self.df = self.df[["Vendedor","Total Vendas Loja 1","Total Vendas Loja 2"]]

        self.df = pd.concat([arquivo1,arquivo2])

        self.df = self.df.drop_duplicates(subset="Id Vendedor")

        self.atualiza_treeview()
        self.soma_colunas_com_valor()

    def merge_left_join(self):
        tipo_de_arquivo = (("Excel files","*.xlsx;*.xls"),("All files","*.*"))

        arquivo1 = filedialog.askopenfilename(title="Selecione o primeiro arquivo",
                                                        filetypes=tipo_de_arquivo)
        arquivo2 = filedialog.askopenfilename(title="Selecione o segundo arquivo",
                                                        filetypes=tipo_de_arquivo) 

        arquivo1 = pd.read_excel(arquivo1)
        arquivo2 = pd.read_excel(arquivo2)

        #coluna_join = simpledialog.askstring("Coluna do Full Join","Digite o nome da coluna que será utilizada para o Full Join: ")
        
        #self.df = pd.merge(arquivo1,arquivo2,on=coluna_join, how="left")
        #self.df = pd.merge(arquivo1,arquivo2[["Vendedor","Total Vendas"]],on=coluna_join, how="inner",suffixes=(" Loja 1"," Loja 2"))
        self.df = pd.merge(arquivo1,arquivo2,on=["Id Vendedor"], how="left",suffixes=(" Vendas"," Checagem"))
        
        self.df = self.df.dropna()

        #self.df = pd.concat([arquivo1,arquivo2])

        #self.df = self.df.drop_duplicates(subset="Id Vendedor")

        self.atualiza_treeview()
        self.soma_colunas_com_valor()

    def merge_outer(self):
        tipo_de_arquivo = (("Excel files","*.xlsx;*.xls"),("All files","*.*"))

        arquivo1 = filedialog.askopenfilename(title="Selecione o primeiro arquivo",
                                                        filetypes=tipo_de_arquivo)
        arquivo2 = filedialog.askopenfilename(title="Selecione o segundo arquivo",
                                                        filetypes=tipo_de_arquivo) 

        arquivo1 = pd.read_excel(arquivo1)
        arquivo2 = pd.read_excel(arquivo2)

        #coluna_join = simpledialog.askstring("Coluna do Outer","Digite o nome da coluna que será utilizada para o Outer: ")
        
        #self.df = pd.merge(arquivo1,arquivo2,on=coluna_join, how="outer")
        #self.df = pd.merge(arquivo1,arquivo2[["Vendedor","Total Vendas"]],on=coluna_join, how="inner",suffixes=(" Loja 1"," Loja 2"))
        self.df = pd.merge(arquivo1,arquivo2,on=["Id Vendedor"], how="outer",suffixes=(" Loja 1"," Loja 2"))
        
        self.df = self.df.dropna()

        del self.df["Vendedor Loja 2"]

        #self.df = pd.concat([arquivo1,arquivo2])

        #self.df = self.df.drop_duplicates(subset="Id Vendedor")

        self.atualiza_treeview()
        self.soma_colunas_com_valor()

    def salvar_como(self):
        tipo_de_arquivo = (("Excel files","*.xlsx;*.xls"),("All files","*.*"))

        filename = filedialog.asksaveasfilename(title="Salvar Como",
                                                        filetypes=tipo_de_arquivo,
                                                        defaultextension=".xlsx")
        if filename:
            self.df.to_excel(filename, index=False)

            messagebox.showinfo("Salvo",f"Salvo com sucesso!")

    
    def editarItens(self,event):
        item = self.tree.selection()[0]

        values = self.tree.item(item, "values")

        janela_edicao = tk.Toplevel(self.janela_principal)
        
        for i,nome_coluna in enumerate(self.df.columns):

            label = tk.Label(janela_edicao,text=nome_coluna,font=("Arial",20))
            label.grid(row=i,column=0)

            entry = tk.Entry(janela_edicao,font=("Arial",20))
            entry.insert(0, values[i])
            entry.grid(row=i,column=1)

        salvar = tk.Button(janela_edicao,text="Salvar",command=lambda:self.salvar_alteracoes(item,janela_edicao))
        salvar.grid(row=len(self.df.columns), column=1)
    
    def salvar_alteracoes(self,item,janela_edicao):
        novo_valores = [child.get() for child in janela_edicao.winfo_children() if isinstance(child, tk.Entry)]

        self.df.loc[item, :] = novo_valores

        self.tree.item(item, values=novo_valores)

        janela_edicao.destroy()

    def consolidar_arquivos(self):

        try:
            caminho_arquivos = filedialog.askdirectory(title="Selecione a pasta")

            list_arquivos = os.listdir(caminho_arquivos)

            lista_caminho_e_arquivos_excel = [caminho_arquivos + '\\' + arquivo for arquivo in list_arquivos if arquivo[-4:] == 'xlsx']

            dados_arquivos = pd.DataFrame()

            for arquivo in lista_caminho_e_arquivos_excel:
                dados = pd.read_excel(arquivo)
                dados_arquivos = pd.concat([dados_arquivos, dados],ignore_index=True)

            self.df = dados_arquivos

        except Exception as e:
            messagebox.showerror("Erro",f"Não foi possivel abrir o arquivo: {e}")
        
        self.atualiza_treeview()
        self.soma_colunas_com_valor()

    def quebrar_arquivo(self):

        import re

        coluna = simpledialog.askstring("Separar em Arquivos", "Informe a coluna que deseja quebrar o arquivo: ")

        if coluna:
            grupos = self.df.groupby(coluna)

            pasta_destino = filedialog.askdirectory(title="Selecione a pasta destino dos arquivos")

            if pasta_destino:

                for valor, grupo in grupos:
                    nome_arquivo = re.sub(r'[^\w\-_.]', '_',f"{coluna}_{valor}.xlsx")

                    caminho_arquivo = os.path.join(pasta_destino, nome_arquivo)

                    grupo.to_excel(caminho_arquivo,index=False)

            messagebox.showinfo("Concluido","Relatórios criados com sucesso!")



editor = ExcelEditor(janela)

janela.mainloop()